In [1]:
import pandas as pd
import numpy as np

In [2]:
campaign_data = pd.DataFrame(pd.read_csv('data/campaign_data.csv'))
item_data = pd.DataFrame(pd.read_csv('data/item_data.csv'))
coupon_item_mapping = pd.DataFrame(pd.read_csv('data/coupon_item_mapping.csv'))
ct = pd.DataFrame(pd.read_csv('data/customer_transaction_data.csv'))
cd = pd.DataFrame(pd.read_csv('data/customer_demographics.csv'))

train = pd.DataFrame(pd.read_csv('data/train.csv'))
test = pd.DataFrame(pd.read_csv('data/test.csv'))

In [3]:
train.shape, test.shape

((78369, 5), (50226, 4))

### merging test and train data

In [4]:
train1 = train.append(test, ignore_index = True, sort=False)
print(train1.shape)

(128595, 5)


In [5]:
print(len(train.customer_id.unique()))
print(len(test.customer_id.unique()))
print(len(train1.customer_id.unique()))

1428
1250
1582


### Feature engineering campaign_data

In [6]:
print(campaign_data.shape)
print(campaign_data.dtypes)
campaign_data.head()

(28, 4)
campaign_id       int64
campaign_type    object
start_date       object
end_date         object
dtype: object


,campaign_id,campaign_type,start_date,end_date
0,24,Y,21/10/13,20/12/13
1,25,Y,21/10/13,22/11/13
2,20,Y,07/09/13,16/11/13
3,23,Y,08/10/13,15/11/13
4,21,Y,16/09/13,18/10/13


In [7]:
campaign_data['start-date'] = pd.to_datetime(campaign_data['start_date'], format='%d/%m/%y', dayfirst=True)
campaign_data['end-date'] = pd.to_datetime(campaign_data['end_date'], format='%d/%m/%y', dayfirst=True)
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date,start-date,end-date
0,24,Y,21/10/13,20/12/13,2013-10-21,2013-12-20
1,25,Y,21/10/13,22/11/13,2013-10-21,2013-11-22
2,20,Y,07/09/13,16/11/13,2013-09-07,2013-11-16
3,23,Y,08/10/13,15/11/13,2013-10-08,2013-11-15
4,21,Y,16/09/13,18/10/13,2013-09-16,2013-10-18


In [8]:
campaign_data['date_diff'] = (campaign_data['end-date'] - campaign_data['start-date']).dt.days
campaign_data.head()

,campaign_id,campaign_type,start_date,end_date,start-date,end-date,date_diff
0,24,Y,21/10/13,20/12/13,2013-10-21,2013-12-20,60
1,25,Y,21/10/13,22/11/13,2013-10-21,2013-11-22,32
2,20,Y,07/09/13,16/11/13,2013-09-07,2013-11-16,70
3,23,Y,08/10/13,15/11/13,2013-10-08,2013-11-15,38
4,21,Y,16/09/13,18/10/13,2013-09-16,2013-10-18,32


In [358]:
campaign_data1 = pd.concat([campaign_data,pd.get_dummies(campaign_data.campaign_type, prefix='campaign_type')], axis=1)
campaign_data1.head()

,campaign_id,campaign_type,start_date,end_date,start-date,end-date,date_diff,campaign_type_X,campaign_type_Y
0,24,Y,21/10/13,20/12/13,2013-10-21,2013-12-20,60,0,1
1,25,Y,21/10/13,22/11/13,2013-10-21,2013-11-22,32,0,1
2,20,Y,07/09/13,16/11/13,2013-09-07,2013-11-16,70,0,1
3,23,Y,08/10/13,15/11/13,2013-10-08,2013-11-15,38,0,1
4,21,Y,16/09/13,18/10/13,2013-09-16,2013-10-18,32,0,1


### merging campaign_data1 in train1

In [362]:
train2 = train1.merge(campaign_data1, on='campaign_id', how='left')
print(train2.shape)
train2.head()

(128595, 13)


,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,start_date,end_date,start-date,end-date,date_diff,campaign_type_X,campaign_type_Y
0,1,13,27,1053,0.0,X,19/05/13,05/07/13,2013-05-19,2013-07-05,47,1,0
1,2,13,116,48,0.0,X,19/05/13,05/07/13,2013-05-19,2013-07-05,47,1,0
2,6,9,635,205,0.0,Y,11/03/13,12/04/13,2013-03-11,2013-04-12,32,0,1
3,7,13,644,1050,0.0,X,19/05/13,05/07/13,2013-05-19,2013-07-05,47,1,0
4,9,8,1017,1489,0.0,X,16/02/13,05/04/13,2013-02-16,2013-04-05,48,1,0


In [363]:
train2 = train2.drop(columns=['start_date','end_date','start-date','end-date','campaign_type'])
train2.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,date_diff,campaign_type_X,campaign_type_Y
0,1,13,27,1053,0.0,47,1,0
1,2,13,116,48,0.0,47,1,0
2,6,9,635,205,0.0,32,0,1
3,7,13,644,1050,0.0,47,1,0
4,9,8,1017,1489,0.0,48,1,0


### merging Item Data in Coupon Item Mapping

In [11]:
print(item_data.shape)
item_data.head()

(74066, 4)


,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [12]:
print(coupon_item_mapping.shape)
coupon_item_mapping.head()

(92663, 2)


,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [13]:
coupon_item_mapping1 = coupon_item_mapping.merge(item_data, on='item_id', how='left')

In [14]:
print(coupon_item_mapping1.shape)
coupon_item_mapping1.head()

(92663, 5)


,coupon_id,item_id,brand,brand_type,category
0,105,37,56,Local,Grocery
1,107,75,56,Local,Grocery
2,494,76,209,Established,Grocery
3,522,77,278,Established,Grocery
4,518,77,278,Established,Grocery


### Feature engineering coupon_item_mapping1

In [15]:
print(len(coupon_item_mapping1.coupon_id.unique()))
print(len(coupon_item_mapping1.item_id.unique()))
print(len(coupon_item_mapping1.brand.unique()))
print(len(coupon_item_mapping1.brand_type.unique()))
print(len(coupon_item_mapping1.category.unique()))

1116
36289
2555
2
17


In [16]:
coupon_item_mapping1.dtypes

coupon_id      int64
item_id        int64
brand          int64
brand_type    object
category      object
dtype: object

In [17]:
for i in ['brand', 'brand_type', 'category']:
    coupon_item_mapping1[i]=coupon_item_mapping1[i].astype('category')

In [18]:
coupon_item_mapping1.dtypes

coupon_id        int64
item_id          int64
brand         category
brand_type    category
category      category
dtype: object

In [19]:
coupon_item_mapping1.isna().sum()

coupon_id     0
item_id       0
brand         0
brand_type    0
category      0
dtype: int64

In [20]:
coupon_item_mapping1.head()

,coupon_id,item_id,brand,brand_type,category
0,105,37,56,Local,Grocery
1,107,75,56,Local,Grocery
2,494,76,209,Established,Grocery
3,522,77,278,Established,Grocery
4,518,77,278,Established,Grocery


In [21]:
counts = coupon_item_mapping1['brand'].value_counts()
counts

56      13540
1        1919
686      1696
602      1621
278      1331
4700     1312
2088     1251
158      1146
1470     1004
1101      978
133       919
328       896
1262      881
673       835
1337      827
866       814
209       805
4412      768
967       724
989       720
1124      703
57        658
946       596
544       563
1075      561
681       543
1041      524
487       516
1587      501
982       485
        ...  
4293        2
4292        2
1269        2
4288        2
4287        2
4285        2
1287        2
4282        2
4283        2
4280        2
4272        2
4270        2
1341        2
4271        2
5472        2
4273        2
207         1
5383        1
427         1
4384        1
4554        1
1047        1
1736        1
2045        1
218         1
4983        1
2001        1
1820        1
5390        1
4395        1
Name: brand, Length: 2555, dtype: int64

In [22]:
coupon_item_mapping1['brand_value_counts'] = coupon_item_mapping1['brand'].map(counts)

In [23]:
coupon_item_mapping1.head()

,coupon_id,item_id,brand,brand_type,category,brand_value_counts
0,105,37,56,Local,Grocery,13540
1,107,75,56,Local,Grocery,13540
2,494,76,209,Established,Grocery,805
3,522,77,278,Established,Grocery,1331
4,518,77,278,Established,Grocery,1331


In [24]:
cut_bins = [1, 1000, 10000, 20000]
pd.cut(coupon_item_mapping1['brand_value_counts'], bins=cut_bins).value_counts()

(1, 1000]         67829
(10000, 20000]    13540
(1000, 10000]     11280
Name: brand_value_counts, dtype: int64

In [25]:
cut_labels = ['small','medium','large']
coupon_item_mapping1['brand_bin'] = pd.cut(coupon_item_mapping1.brand_value_counts, 
                                                             bins=cut_bins, 
                                                             labels=cut_labels)

In [26]:
coupon_item_mapping1.head()

,coupon_id,item_id,brand,brand_type,category,brand_value_counts,brand_bin
0,105,37,56,Local,Grocery,13540,large
1,107,75,56,Local,Grocery,13540,large
2,494,76,209,Established,Grocery,805,small
3,522,77,278,Established,Grocery,1331,medium
4,518,77,278,Established,Grocery,1331,medium


In [27]:
cat_value_counts = coupon_item_mapping1['category'].value_counts()
cat_value_counts

Grocery                   36466
Pharmaceutical            25061
Natural Products           6819
Meat                       6218
Packaged Meat              6144
Skin & Hair Care           4924
Seafood                    2227
Flowers & Plants           1963
Dairy, Juices & Snacks     1867
Garden                      286
Prepared Food               240
Miscellaneous               184
Bakery                      100
Salads                      100
Travel                       44
Vegetables (cut)             19
Restauarant                   1
Name: category, dtype: int64

In [28]:
coupon_item_mapping1['category_value_counts'] = coupon_item_mapping1['category'].map(cat_value_counts)

In [29]:
coupon_item_mapping1.head()

,coupon_id,item_id,brand,brand_type,category,brand_value_counts,brand_bin,category_value_counts
0,105,37,56,Local,Grocery,13540,large,36466
1,107,75,56,Local,Grocery,13540,large,36466
2,494,76,209,Established,Grocery,805,small,36466
3,522,77,278,Established,Grocery,1331,medium,36466
4,518,77,278,Established,Grocery,1331,medium,36466


In [30]:
cat_cut_labels = ['others','pharmaceutical','grocery']
cat_cut_bins = [1, 6819, 25061, 36466]
coupon_item_mapping1['category_bin'] = pd.cut(coupon_item_mapping1.category_value_counts, 
                                              bins=cat_cut_bins, 
                                              labels=cat_cut_labels)

In [31]:
coupon_item_mapping1.head()

,coupon_id,item_id,brand,brand_type,category,brand_value_counts,brand_bin,category_value_counts,category_bin
0,105,37,56,Local,Grocery,13540,large,36466,grocery
1,107,75,56,Local,Grocery,13540,large,36466,grocery
2,494,76,209,Established,Grocery,805,small,36466,grocery
3,522,77,278,Established,Grocery,1331,medium,36466,grocery
4,518,77,278,Established,Grocery,1331,medium,36466,grocery


In [32]:
coupon_item_mapping1 = coupon_item_mapping1.drop(columns=['brand','category','brand_value_counts','category_value_counts'])

In [127]:
coupon_item_mapping1.head()

,coupon_id,item_id,brand_type,brand_bin,category_bin
0,105,37,Local,large,grocery
1,107,75,Local,large,grocery
2,494,76,Established,small,grocery
3,522,77,Established,medium,grocery
4,518,77,Established,medium,grocery


In [320]:
coupon_item_mapping2 = pd.concat([coupon_item_mapping1,
                                  pd.get_dummies(coupon_item_mapping1.brand_type, prefix='type')],
                                 axis=1)
coupon_item_mapping2.head()

,coupon_id,item_id,brand_type,brand_bin,category_bin,type_Established,type_Local
0,105,37,Local,large,grocery,0,1
1,107,75,Local,large,grocery,0,1
2,494,76,Established,small,grocery,1,0
3,522,77,Established,medium,grocery,1,0
4,518,77,Established,medium,grocery,1,0


In [321]:
coupon_item_mapping3 = pd.concat([coupon_item_mapping2, pd.get_dummies(coupon_item_mapping2.brand_bin, prefix='brand')],
                                 axis=1)
coupon_item_mapping3.head()

,coupon_id,item_id,brand_type,brand_bin,category_bin,type_Established,type_Local,brand_small,brand_medium,brand_large
0,105,37,Local,large,grocery,0,1,0,0,1
1,107,75,Local,large,grocery,0,1,0,0,1
2,494,76,Established,small,grocery,1,0,1,0,0
3,522,77,Established,medium,grocery,1,0,0,1,0
4,518,77,Established,medium,grocery,1,0,0,1,0


In [322]:
coupon_item_mapping4=pd.concat([coupon_item_mapping3,pd.get_dummies(coupon_item_mapping3.category_bin,prefix='cat')],
                                 axis=1)
coupon_item_mapping4.head()

,coupon_id,item_id,brand_type,brand_bin,category_bin,type_Established,type_Local,brand_small,brand_medium,brand_large,cat_others,cat_pharmaceutical,cat_grocery
0,105,37,Local,large,grocery,0,1,0,0,1,0,0,1
1,107,75,Local,large,grocery,0,1,0,0,1,0,0,1
2,494,76,Established,small,grocery,1,0,1,0,0,0,0,1
3,522,77,Established,medium,grocery,1,0,0,1,0,0,0,1
4,518,77,Established,medium,grocery,1,0,0,1,0,0,0,1


In [323]:
coupon_item_mapping5 = coupon_item_mapping4.drop(columns=['brand_type','brand_bin','category_bin'])
coupon_item_mapping5.head()

,coupon_id,item_id,type_Established,type_Local,brand_small,brand_medium,brand_large,cat_others,cat_pharmaceutical,cat_grocery
0,105,37,0,1,0,0,1,0,0,1
1,107,75,0,1,0,0,1,0,0,1
2,494,76,1,0,1,0,0,0,0,1
3,522,77,1,0,0,1,0,0,0,1
4,518,77,1,0,0,1,0,0,0,1


### merging coupon_item_mapping5 in train2

In [365]:
print(train2.shape)
train2.head()

(128595, 8)


,id,campaign_id,coupon_id,customer_id,redemption_status,date_diff,campaign_type_X,campaign_type_Y
0,1,13,27,1053,0.0,47,1,0
1,2,13,116,48,0.0,47,1,0
2,6,9,635,205,0.0,32,0,1
3,7,13,644,1050,0.0,47,1,0
4,9,8,1017,1489,0.0,48,1,0


In [366]:
print(len(coupon_item_mapping5.coupon_id.unique()))
print(len(train2.coupon_id.unique()))

1116
1116


In [367]:
coupon_item_mapping5[coupon_item_mapping5['coupon_id'] == 109].head(1)

,coupon_id,item_id,type_Established,type_Local,brand_small,brand_medium,brand_large,cat_others,cat_pharmaceutical,cat_grocery
796,109,3309,0,1,0,0,1,0,0,1


In [368]:
train3 = train2.merge(coupon_item_mapping5.groupby(['coupon_id']).agg({
    'type_Established':'first',
    'type_Local':'first',
    'brand_small':'first',
    'brand_medium':'first',
    'brand_large':'first',
    'cat_others':'first',
    'cat_pharmaceutical':'first',
    'cat_grocery':'first'
}).reset_index(), on='coupon_id', how='left')

In [369]:
print(train3.shape)
train3.head()

(128595, 16)


,id,campaign_id,coupon_id,customer_id,redemption_status,date_diff,campaign_type_X,campaign_type_Y,type_Established,type_Local,brand_small,brand_medium,brand_large,cat_others,cat_pharmaceutical,cat_grocery
0,1,13,27,1053,0.0,47,1,0,1,0,1,0,0,0,0,1
1,2,13,116,48,0.0,47,1,0,0,1,0,0,1,0,0,1
2,6,9,635,205,0.0,32,0,1,1,0,1,0,0,0,1,0
3,7,13,644,1050,0.0,47,1,0,1,0,1,0,0,0,0,1
4,9,8,1017,1489,0.0,48,1,0,1,0,1,0,0,0,0,1


In [370]:
print(len(train3.coupon_id.unique()))

1116


### Feature engineering customer_transaction

In [164]:
ct = pd.DataFrame(pd.read_csv('data/customer_transaction_data.csv'))

In [165]:
print(ct.shape)
ct.head()

(1324566, 7)


,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [166]:
ct['date'] = pd.to_datetime(ct['date'], format='%Y-%m-%d')
# fetching day of the month and whether the day is a weekend or not
ct['dayofmonth'] = ct['date'].dt.day
ct['weekend'] = ct['date'].dt.dayofweek

In [167]:
#if weekend then 1 , 0 otherwise, (Monday=0, Sunday=6)
ct['weekend'] = (ct['weekend'] > 4).astype(int)

In [168]:
ct.tail()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,dayofmonth,weekend
1324561,2013-06-30,1129,2777,1,284.60,-71.24,0.0,30,1
1324562,2013-06-30,1129,2953,4,42.74,-28.50,0.0,30,1
1324563,2013-06-30,1129,2971,6,64.12,-42.74,0.0,30,1
1324564,2013-06-30,1129,46984,1,95.82,0.00,0.0,30,1
1324565,2013-06-30,1129,64498,2,489.78,0.00,0.0,30,1


In [169]:
ct['total_discount'] = ct['other_discount'] + ct['coupon_discount']
ct['cost_price'] = ct['selling_price'] - ct['total_discount']
ct['percent_discount'] = (ct['total_discount'] / ct['cost_price'])
ct.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,dayofmonth,weekend,total_discount,cost_price,percent_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,2,0,-10.69,45.95,-0.232644
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,2,0,-13.89,67.32,-0.206328
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,2,0,-14.25,120.75,-0.118012
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,2,0,0.00,67.32,0.000000
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,2,0,-28.14,99.38,-0.283156


In [170]:
ct['coupon_disc_bin'] = ct['coupon_discount'].apply(lambda x:0 if x>=0 else 1)

In [171]:
ct[ct['customer_id']==464].head(1)

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,dayofmonth,weekend,total_discount,cost_price,percent_discount,coupon_disc_bin
88,2012-01-02,464,5525,1,106.5,-35.62,-35.62,2,0,-71.24,177.74,-0.40081,1


### merging ct in train3

In [371]:
print(train3.shape)
train3.head()

(128595, 16)


,id,campaign_id,coupon_id,customer_id,redemption_status,date_diff,campaign_type_X,campaign_type_Y,type_Established,type_Local,brand_small,brand_medium,brand_large,cat_others,cat_pharmaceutical,cat_grocery
0,1,13,27,1053,0.0,47,1,0,1,0,1,0,0,0,0,1
1,2,13,116,48,0.0,47,1,0,0,1,0,0,1,0,0,1
2,6,9,635,205,0.0,32,0,1,1,0,1,0,0,0,1,0
3,7,13,644,1050,0.0,47,1,0,1,0,1,0,0,0,0,1
4,9,8,1017,1489,0.0,48,1,0,1,0,1,0,0,0,0,1


In [372]:
print(ct.shape)
ct.head()

(1324566, 13)


,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,dayofmonth,weekend,total_discount,cost_price,percent_discount,coupon_disc_bin
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,2,0,-10.69,45.95,-0.232644,0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,2,0,-13.89,67.32,-0.206328,0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,2,0,-14.25,120.75,-0.118012,0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,2,0,0.00,67.32,0.000000,0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,2,0,-28.14,99.38,-0.283156,0


In [373]:
print(len(ct.customer_id.unique()))
print(len(train3.customer_id.unique()))

1582
1582


In [406]:
### group all columns in ct1 wrt unique customer_id
train4=train3.merge(ct.groupby(['customer_id']).agg({
                                                'item_id':'count','quantity':sum,'selling_price':sum,
                                                'other_discount':sum,'coupon_discount':sum,'dayofmonth':'first',
                                                'weekend':'first','total_discount':sum,'cost_price':sum,
                                                'percent_discount':sum,'coupon_disc_bin':'first'
                                                
                                                }).reset_index(),on=['customer_id'],how='left')
#'date':'first'

In [407]:
train4.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,date_diff,campaign_type_X,campaign_type_Y,type_Established,type_Local,...,quantity,selling_price,other_discount,coupon_discount,dayofmonth,weekend,total_discount,cost_price,percent_discount,coupon_disc_bin
0,1,13,27,1053,0.0,47,1,0,1,0,...,105551,57120.75,-10282.37,-89.05,10,0,-10371.42,67492.17,-46.472834,0
1,2,13,116,48,0.0,47,1,0,0,1,...,12143,90185.10,-10664.18,-1237.79,4,1,-11901.97,102087.07,-60.020278,0
2,6,9,635,205,0.0,32,0,1,1,0,...,1351,117461.66,-17261.79,-2145.72,1,0,-19407.51,136869.17,-118.447682,0
3,7,13,644,1050,0.0,47,1,0,1,0,...,306,23291.42,-3947.37,-178.10,14,0,-4125.47,27416.89,-32.377078,0
4,9,8,1017,1489,0.0,48,1,0,1,0,...,139063,67797.49,-11534.90,-265.01,26,0,-11799.91,79597.40,-77.654963,0


In [408]:
train4.shape

(128595, 27)

In [ ]:
#rename(                                             
#                                                columns={                                                
#                                                    'quantity':'quantity_count',
#                                                    'percent_discount':'percent_discount_sum',
#                                                    'coupon_id':'coupon_id_count',
#                                                    'item_id':'item_id_count'})

In [ ]:
#ct3 = ct2.groupby(['customer_id']).agg({'item_id':'count',
#                                       })

In [ ]:
#ct2 = ct1.drop(columns=['selling_price','other_discount','coupon_discount',''])

### feature engineering cd

In [277]:
cd = pd.DataFrame(pd.read_csv('data/customer_demographics.csv'))

In [278]:
print(cd.shape)
cd.head()

(760, 7)


,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [279]:
cd.isna().sum()

customer_id         0
age_range           0
marital_status    329
rented              0
family_size         0
no_of_children    538
income_bracket      0
dtype: int64

In [280]:
# percentage if missing values
cd.isnull().sum() * 100 / len(cd)

customer_id        0.000000
age_range          0.000000
marital_status    43.289474
rented             0.000000
family_size        0.000000
no_of_children    70.789474
income_bracket     0.000000
dtype: float64

In [281]:
#marital status : Married=1, Single=0, NaN=-1
cd['marital_status'] = cd['marital_status'].replace({'Married':1, 'Single':0, np.nan:-1})
cd.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,1,0,2,NaN,4
1,6,46-55,1,0,2,NaN,5
2,7,26-35,-1,0,3,1,3
3,8,26-35,-1,0,4,2,6
4,10,46-55,0,0,1,NaN,5


In [282]:
a = cd['age_range'].str.findall('(\d+)')
a.head()

0        [70]
1    [46, 55]
2    [26, 35]
3    [26, 35]
4    [46, 55]
Name: age_range, dtype: object

In [283]:
b = pd.Series([])
for i in range(0,len(a)):
    a[i] = np.array(a[i]).astype(np.float)
    b[i] = sum(a[i]) / len(a[i])
b.head()

0    70.0
1    50.5
2    30.5
3    30.5
4    50.5
dtype: float64

In [284]:
cd['age'] = b
cd.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket,age
0,1,70+,1,0,2,NaN,4,70.0
1,6,46-55,1,0,2,NaN,5,50.5
2,7,26-35,-1,0,3,1,3,30.5
3,8,26-35,-1,0,4,2,6,30.5
4,10,46-55,0,0,1,NaN,5,50.5


In [287]:
cd.family_size.unique()

array(['2', '3', '4', '1', 5], dtype=object)

In [288]:
cd['family_size'] = cd['family_size'].replace({'5+':5})
cd.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket,age
0,1,70+,1,0,2,NaN,4,70.0
1,6,46-55,1,0,2,NaN,5,50.5
2,7,26-35,-1,0,3,1,3,30.5
3,8,26-35,-1,0,4,2,6,30.5
4,10,46-55,0,0,1,NaN,5,50.5


In [289]:
cd['family_size'] = cd['family_size'].astype('int64')
cd.family_size.value_counts(bins=3)

(0.995, 2.333]    551
(3.667, 5.0]      105
(2.333, 3.667]    104
Name: family_size, dtype: int64

In [290]:
cut_bins_family = [0, 2, 4, 5]
cut_labels_family = ['childless', 'nuclear', 'extended']
cd['family_bin'] = pd.cut(cd['family_size'], bins=cut_bins_family, labels=cut_labels_family)

In [291]:
cd.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket,age,family_bin
0,1,70+,1,0,2,NaN,4,70.0,childless
1,6,46-55,1,0,2,NaN,5,50.5,childless
2,7,26-35,-1,0,3,1,3,30.5,nuclear
3,8,26-35,-1,0,4,2,6,30.5,nuclear
4,10,46-55,0,0,1,NaN,5,50.5,childless


In [292]:
cd['no_of_children'] = cd['no_of_children'].replace({'3+':3, np.nan:0})
cd.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket,age,family_bin
0,1,70+,1,0,2,0,4,70.0,childless
1,6,46-55,1,0,2,0,5,50.5,childless
2,7,26-35,-1,0,3,1,3,30.5,nuclear
3,8,26-35,-1,0,4,2,6,30.5,nuclear
4,10,46-55,0,0,1,0,5,50.5,childless


In [293]:
cd['no_of_children'] = cd['no_of_children'].fillna(0)
cd.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket,age,family_bin
0,1,70+,1,0,2,0,4,70.0,childless
1,6,46-55,1,0,2,0,5,50.5,childless
2,7,26-35,-1,0,3,1,3,30.5,nuclear
3,8,26-35,-1,0,4,2,6,30.5,nuclear
4,10,46-55,0,0,1,0,5,50.5,childless


In [294]:
cd.dtypes

customer_id          int64
age_range           object
marital_status       int64
rented               int64
family_size          int64
no_of_children      object
income_bracket       int64
age                float64
family_bin        category
dtype: object

In [295]:
cd['no_of_children'] = cd['no_of_children'].astype('int64')

In [296]:
cd['children_bin'] = cd['no_of_children'].apply(lambda x:0 if x>0 else 1)
cd.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket,age,family_bin,children_bin
0,1,70+,1,0,2,0,4,70.0,childless,1
1,6,46-55,1,0,2,0,5,50.5,childless,1
2,7,26-35,-1,0,3,1,3,30.5,nuclear,0
3,8,26-35,-1,0,4,2,6,30.5,nuclear,0
4,10,46-55,0,0,1,0,5,50.5,childless,1


In [297]:
print(cd.income_bracket.unique())
cd.income_bracket.value_counts(bins=3)

[ 4  5  3  6  1  7  2  8  9 12 10 11]


(0.988, 4.667]    362
(4.667, 8.333]    344
(8.333, 12.0]      54
Name: income_bracket, dtype: int64

In [298]:
cut_labels_income = ['lower','middle','upper']
cd['income_range'] = pd.cut(cd.income_bracket, bins=3, labels=cut_labels_income)

In [299]:
cd.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket,age,family_bin,children_bin,income_range
0,1,70+,1,0,2,0,4,70.0,childless,1,lower
1,6,46-55,1,0,2,0,5,50.5,childless,1,middle
2,7,26-35,-1,0,3,1,3,30.5,nuclear,0,lower
3,8,26-35,-1,0,4,2,6,30.5,nuclear,0,middle
4,10,46-55,0,0,1,0,5,50.5,childless,1,middle


In [302]:
cd1 = pd.concat([cd, pd.get_dummies(cd.income_range)], axis=1)
cd1.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket,age,family_bin,children_bin,income_range,lower,middle,upper
0,1,70+,1,0,2,0,4,70.0,childless,1,lower,1,0,0
1,6,46-55,1,0,2,0,5,50.5,childless,1,middle,0,1,0
2,7,26-35,-1,0,3,1,3,30.5,nuclear,0,lower,1,0,0
3,8,26-35,-1,0,4,2,6,30.5,nuclear,0,middle,0,1,0
4,10,46-55,0,0,1,0,5,50.5,childless,1,middle,0,1,0


In [303]:
cd2 = pd.concat([cd1, pd.get_dummies(cd1.family_bin)], axis=1)
cd2.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket,age,family_bin,children_bin,income_range,lower,middle,upper,childless,nuclear,extended
0,1,70+,1,0,2,0,4,70.0,childless,1,lower,1,0,0,1,0,0
1,6,46-55,1,0,2,0,5,50.5,childless,1,middle,0,1,0,1,0,0
2,7,26-35,-1,0,3,1,3,30.5,nuclear,0,lower,1,0,0,0,1,0
3,8,26-35,-1,0,4,2,6,30.5,nuclear,0,middle,0,1,0,0,1,0
4,10,46-55,0,0,1,0,5,50.5,childless,1,middle,0,1,0,1,0,0


In [304]:
cd3 = cd2.drop(columns=['age_range','no_of_children','income_bracket','income_range','family_bin'])
cd3.head()

,customer_id,marital_status,rented,family_size,age,children_bin,lower,middle,upper,childless,nuclear,extended
0,1,1,0,2,70.0,1,1,0,0,1,0,0
1,6,1,0,2,50.5,1,0,1,0,1,0,0
2,7,-1,0,3,30.5,0,1,0,0,0,1,0
3,8,-1,0,4,30.5,0,0,1,0,0,1,0
4,10,0,0,1,50.5,1,0,1,0,1,0,0


### merging cd3 in train4

In [409]:
print(len(cd1.customer_id.unique()))
print(len(train4.customer_id.unique()))

760
1582


In [410]:
train5 = train4.merge(cd3, on='customer_id', how='left')

In [411]:
train5.shape

(128595, 38)

In [412]:
train5.isna().sum()

id                        0
campaign_id               0
coupon_id                 0
customer_id               0
redemption_status     50226
date_diff                 0
campaign_type_X           0
campaign_type_Y           0
type_Established          0
type_Local                0
brand_small               0
brand_medium              0
brand_large               0
cat_others                0
cat_pharmaceutical        0
cat_grocery               0
item_id                   0
quantity                  0
selling_price             0
other_discount            0
coupon_discount           0
dayofmonth                0
weekend                   0
total_discount            0
cost_price                0
percent_discount          0
coupon_disc_bin           0
marital_status        53995
rented                53995
family_size           53995
age                   53995
children_bin          53995
lower                 53995
middle                53995
upper                 53995
childless           

In [413]:
train5['marital_status'] = train5['marital_status'].fillna(-1)
train5['rented'] = train5['rented'].fillna(-1)
train5['family_size'] = train5['family_size'].fillna(0)
train5['age'] = train5['age'].fillna(-1)
train5['children_bin'] = train5['children_bin'].fillna(0)

train5['lower'] = train5['lower'].fillna(0)
train5['middle'] = train5['middle'].fillna(0)
train5['upper'] = train5['upper'].fillna(0)
train5['childless'] = train5['childless'].fillna(0)
train5['nuclear'] = train5['nuclear'].fillna(0)
train5['extended'] = train5['extended'].fillna(0)

In [414]:
train5.isna().sum()

id                        0
campaign_id               0
coupon_id                 0
customer_id               0
redemption_status     50226
date_diff                 0
campaign_type_X           0
campaign_type_Y           0
type_Established          0
type_Local                0
brand_small               0
brand_medium              0
brand_large               0
cat_others                0
cat_pharmaceutical        0
cat_grocery               0
item_id                   0
quantity                  0
selling_price             0
other_discount            0
coupon_discount           0
dayofmonth                0
weekend                   0
total_discount            0
cost_price                0
percent_discount          0
coupon_disc_bin           0
marital_status            0
rented                    0
family_size               0
age                       0
children_bin              0
lower                     0
middle                    0
upper                     0
childless           

In [415]:
train5.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,date_diff,campaign_type_X,campaign_type_Y,type_Established,type_Local,...,rented,family_size,age,children_bin,lower,middle,upper,childless,nuclear,extended
0,1,13,27,1053,0.0,47,1,0,1,0,...,0.0,1.0,50.5,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,2,13,116,48,0.0,47,1,0,0,1,...,0.0,2.0,40.5,1.0,1.0,0.0,0.0,1.0,0.0,0.0
2,6,9,635,205,0.0,32,0,1,1,0,...,0.0,2.0,50.5,1.0,0.0,1.0,0.0,1.0,0.0,0.0
3,7,13,644,1050,0.0,47,1,0,1,0,...,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9,8,1017,1489,0.0,48,1,0,1,0,...,0.0,2.0,50.5,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [416]:
train5.shape

(128595, 38)

In [417]:
train5.dtypes

id                      int64
campaign_id             int64
coupon_id               int64
customer_id             int64
redemption_status     float64
date_diff               int64
campaign_type_X         uint8
campaign_type_Y         uint8
type_Established        uint8
type_Local              uint8
brand_small             uint8
brand_medium            uint8
brand_large             uint8
cat_others              uint8
cat_pharmaceutical      uint8
cat_grocery             uint8
item_id                 int64
quantity                int64
selling_price         float64
other_discount        float64
coupon_discount       float64
dayofmonth              int64
weekend                 int32
total_discount        float64
cost_price            float64
percent_discount      float64
coupon_disc_bin         int64
marital_status        float64
rented                float64
family_size           float64
age                   float64
children_bin          float64
lower                 float64
middle    

In [418]:
final_train = train5[train5['redemption_status'].isnull() == False].copy()
final_test = train5[train5['redemption_status'].isnull() == True].copy()
print(final_train.shape,final_test.shape)

(78369, 38) (50226, 38)


In [419]:
final_train.columns

Index(['id', 'campaign_id', 'coupon_id', 'customer_id', 'redemption_status',
       'date_diff', 'campaign_type_X', 'campaign_type_Y', 'type_Established',
       'type_Local', 'brand_small', 'brand_medium', 'brand_large',
       'cat_others', 'cat_pharmaceutical', 'cat_grocery', 'item_id',
       'quantity', 'selling_price', 'other_discount', 'coupon_discount',
       'dayofmonth', 'weekend', 'total_discount', 'cost_price',
       'percent_discount', 'coupon_disc_bin', 'marital_status', 'rented',
       'family_size', 'age', 'children_bin', 'lower', 'middle', 'upper',
       'childless', 'nuclear', 'extended'],
      dtype='object')

In [420]:
final_train.to_csv('training_data.csv', index=False)
final_test.to_csv('testing_data.csv', index=False)